In [0]:
import numpy as np
import librosa

from torch.autograd import Variable
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from sklearn.model_selection import train_test_split


# Interface colab notebook with dataset in google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Check google drive directory 
!ls "/content/drive/My Drive/Colab Notebooks/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

 NN-MFCCs.ipynb   NonProg  'Progressive Rock Songs'


# Get mp3 file names

In [0]:
from google.colab import drive

import glob

filePaths_NonProg = glob.glob("/content/drive/My Drive/Colab Notebooks/NonProg/***.mp3")
filePaths_Prog = glob.glob("/content/drive/My Drive/Colab Notebooks/Progressive Rock Songs/***.mp3")
print(len(filePaths_NonProg))
print(len(filePaths_Prog))
numberNonProg = len(filePaths_NonProg)
numberProg = len(filePaths_Prog)
#for i in range(len(filePaths_Prog)):
#  print(filePaths_Prog[i])

302
73


# Extract Features - MFCCs and Covariance of MFCCs

In [0]:

numOfMFCCS = 20

def getMFCCS(x, sr, nMFCCs):
    stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
    mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
    meanMFCCS = np.mean(mfccs, axis=1)

    cov = np.cov(mfccs)
    upperCovIndicies = np.triu_indices(nMFCCs)
    upperCov = cov[upperCovIndicies]
    return meanMFCCS, upperCov, mfccs

def normalizeInputs(x):
    mean = x.mean()
    std = x.std()
    z = (x-mean)/std
    return z

Grab features from files, store into matrix

In [0]:
inputSize = 230
nonProgInputs = np.zeros((numberNonProg, inputSize))
for i, file in enumerate(filePaths_NonProg):
  print(i)
  x, sr = librosa.load(file, sr=None, mono=True, duration = 40)
  meanMFCC, covMFCC, mfccs = getMFCCS(x, sr, numOfMFCCS)
  meanMFCCNormalized = normalizeInputs(meanMFCC)
  covMFCCNormalized = normalizeInputs(covMFCC)
  
  inputs = np.concatenate((meanMFCCNormalized, covMFCCNormalized))
  inputs = np.expand_dims(inputs,axis=1).T

  nonProgInputs[i,:] = inputs
  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
progInputs = np.zeros((numberProg, inputSize))
for i, file in enumerate(filePaths_Prog):
  print(i)
  x, sr = librosa.load(file, sr=None, mono=True, duration = 40)
  meanMFCC, covMFCC, mfccs = getMFCCS(x, sr, numOfMFCCS)
  meanMFCCNormalized = normalizeInputs(meanMFCC)
  covMFCCNormalized = normalizeInputs(covMFCC)
  
  inputs = np.concatenate((meanMFCCNormalized, covMFCCNormalized))
  inputs = np.expand_dims(inputs,axis=1).T

  progInputs[i,:] = inputs




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


In [0]:
#Combine prog with non prog inputs
allInputs = np.concatenate((progInputs, nonProgInputs))
allInputs.shape

(375, 230)

# Create data labels

In [0]:
#Create labels 
yProg = np.ones((numberProg,1))
yNonProg = -np.ones((numberNonProg,1))
y = np.concatenate((yProg, yNonProg))

# Split dataset into training set and test set

In [0]:
#splitting the training set
testSize = .2
randStateSplit = 42
inputTrain, inputTest, labelTrain, labelTest = train_test_split(allInputs, y, test_size=testSize, random_state = randStateSplit, shuffle = True)

# Convert our dataset into a pseudo-pytorch data loader format to feed into the Neural Network (can be improved)

In [0]:

numTrainingSamples = inputTrain.shape[0]
input_size = inputTrain.shape[1]

batch_size = 25
numOfTrainingBatches = 12

inputTrainPD = pd.DataFrame(np.concatenate((labelTrain, inputTrain),axis = 1))

train_loader = []
labels = []
for g, temp_df in inputTrainPD.groupby(np.arange(numTrainingSamples) // batch_size):
  tempLabels = np.asarray(temp_df.iloc[:,0])
  tempLabels = torch.LongTensor(tempLabels)
  tempLabelsSqueezed = torch.squeeze(tempLabels)
  labels.append(tempLabelsSqueezed)

  tempInputs = temp_df.iloc[:,1:].as_matrix() #convert to np array ... as_matrix returns np array
  torchSamples = []
  for i in range(batch_size):
    #import pdb; pdb.set_trace()
    tempTorchSample = torch.FloatTensor(tempInputs[i])
    torchSamples.append(torch.t(tempTorchSample.view(input_size,1)))
  #   import pdb; pdb.set_trace()
  stackedInputTensors = torch.stack(torchSamples)    

  train_loader.append(stackedInputTensors.view(batch_size,input_size))

# Neural Network Architecture

In [0]:
hidden_size1 = 200
hidden_size2 = 50
num_classes = 1
learning_rate = .001
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2) 
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

net = NeuralNet(input_size, hidden_size1, hidden_size2, num_classes).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  



# Train model

In [0]:
num_epochs = 100
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i in range(numOfTrainingBatches):  
        # Move tensors to the configured device
        input_x = Variable(train_loader[i])
        label_x = Variable(labels[i])
        
        # Forward pass
        outputs = net(input_x.float())
        outputs = torch.squeeze(outputs)
        
        #import pdb; pdb.set_trace()
        loss = criterion(outputs, label_x.float())
        if loss <= 0.008:
          break
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #if (i+1) % 100 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/100], Step [1/12], Loss: 0.9715
Epoch [1/100], Step [2/12], Loss: 0.7844
Epoch [1/100], Step [3/12], Loss: 0.6394
Epoch [1/100], Step [4/12], Loss: 0.6834
Epoch [1/100], Step [5/12], Loss: 0.4166
Epoch [1/100], Step [6/12], Loss: 0.4544
Epoch [1/100], Step [7/12], Loss: 0.9091
Epoch [1/100], Step [8/12], Loss: 0.7093
Epoch [1/100], Step [9/12], Loss: 0.8139
Epoch [1/100], Step [10/12], Loss: 0.8059
Epoch [1/100], Step [11/12], Loss: 0.7959
Epoch [1/100], Step [12/12], Loss: 0.3002
Epoch [2/100], Step [1/12], Loss: 0.7733
Epoch [2/100], Step [2/12], Loss: 0.5200
Epoch [2/100], Step [3/12], Loss: 0.4285
Epoch [2/100], Step [4/12], Loss: 0.6493
Epoch [2/100], Step [5/12], Loss: 0.3370
Epoch [2/100], Step [6/12], Loss: 0.4312
Epoch [2/100], Step [7/12], Loss: 0.8026
Epoch [2/100], Step [8/12], Loss: 0.6258
Epoch [2/100], Step [9/12], Loss: 0.7216
Epoch [2/100], Step [10/12], Loss: 0.7331
Epoch [2/100], Step [11/12], Loss: 0.7125
Epoch [2/100], Step [12/12], Loss: 0.3453
Epoch [3/1

# Test model

In [0]:
correct = 0
testSize = inputTest.shape[0]
predictions = []

for i in range(testSize):
    #import pdb; pdb.set_trace()
    tempInputs = torch.FloatTensor(inputTest[i])
    output = net(Variable(tempInputs))
    if output.data[0] <= 0:
      predicted = -1.
    else:
      predicted = 1.
    predictions.append(predicted)
    correct += (predicted == labelTest[i][0])

print('Accuracy of the network: %d %%' % (100 * correct / testSize))

# Save the Model
#torch.save(net.state_dict(), 'model.pkl')



Accuracy of the network: 80 %


# Save the NN weights

In [0]:
# Save the model checkpoint
#torch.save(net.state_dict(), 'NN80PercentAccuracy.ckpt')

import pickle
with open('NN80PercentTestAccuracy.ckpt', 'wb') as f:
    pickle.dump(net.state_dict(), f)
    